# MNIST를 통한 NN 실습

In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [6]:
tf.reset_default_graph()     #에러가 나면 얘 한번 실행시켜줘

one hot incoding  
        0->[1,0,0,0,0,0,0,0,0,0,0]  
        1->[0,1,0,0,0,0,0,0,0,0,0]  
        2->[0,0,1,0,0,0,0,0,0,0,0]  
        .  
        .  
        .  
        

In [7]:
mnist = input_data.read_data_sets('data/mnist/', one_hot=True) 
mnist

W0624 17:44:10.272722  2088 deprecation.py:323] From <ipython-input-7-81a7507aa580>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0624 17:44:10.274722  2088 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\finance\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0624 17:44:10.275722  2088 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\finance\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instruc

Extracting data/mnist/train-images-idx3-ubyte.gz


W0624 17:44:10.480730  2088 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\finance\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0624 17:44:10.482730  2088 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\finance\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0624 17:44:10.522732  2088 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\finance\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecat

Extracting data/mnist/train-labels-idx1-ubyte.gz
Extracting data/mnist/t10k-images-idx3-ubyte.gz
Extracting data/mnist/t10k-labels-idx1-ubyte.gz


Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x0000000009DAAE80>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000000001200A860>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x000000001200A828>)

In [8]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

Learning_rate, epoches, batch_size, optimizer를 바꿔가면서 학습시켜봅시다.

In [9]:
learning_rate = 0.001
epoches = 20
batch_size = 100

# w = tf.Variable(tf.random_normal([784, 10])) # 784 = 28*28
# b = tf.Variable(tf.random_normal([10]))
keep_prob = tf.placeholder(dtype=tf.float32)

w1 = tf.get_variable("w1", shape=[784, 256],
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.get_variable("b1", shape=[256], 
                     initializer=tf.contrib.layers.xavier_initializer())
l1 = tf.nn.relu(tf.matmul(x, w1) + b1) #non_linear의 중요성, sigmoid보다 relu가 정확
l1 = tf.nn.dropout(l1, keep_prob=keep_prob)

w2 = tf.get_variable("w2", shape=[256, 128],  # depth와 width를 정해주면 그에 weight의 개수가 정해짐, hypo의 복잡도만 우리가 정해주면 됨. 그런데 255든 256이든 별 차이 없어, 얘는 256에서 128로 조정해준거
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.get_variable("b2", shape=[128], 
                     initializer=tf.contrib.layers.xavier_initializer())
l2 = tf.nn.relu(tf.matmul(l1, w2) + b2)
l2 = tf.nn.dropout(l2, keep_prob=keep_prob)

w3 = tf.get_variable("w3", shape=[128, 10],
                     initializer=tf.contrib.layers.xavier_initializer()) #xavier : restrict boltsman machin보다 더 영리!
b3 =  tf.get_variable("b3", shape=[10], 
                     initializer=tf.contrib.layers.xavier_initializer())
hypo = tf.matmul(l2, w3) + b3 # = hypo = wx + b
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(      #logit 손실
        logits=hypo, labels=tf.stop_gradient(y)    #대신 logits에 예측치,
                                                    #labels에다가 실제값
    )
)

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost) # grand 대신 아담 옵티마이저로 바꿔줌

W0624 17:44:10.802140  2088 deprecation.py:506] From <ipython-input-9-e470fc512f0b>:14: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0624 17:44:10.919945  2088 deprecation.py:323] From <ipython-input-9-e470fc512f0b>:30: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [10]:
import time

In [11]:
start = time.time()

sess = tf.Session()               #세션 열고
sess.run(tf.global_variables_initializer())     #시작, 러닝시킴

total_batch = mnist.train.num_examples//batch_size
print('total_batch : ', total_batch)

for epoch in range(epoches):       #에폽(전체를 학습을 시키는거 몇번 돌거야?-> 학습의 정도가 달라짐.)만큼 돌다가     보통 10번 ~ 30번 정도 돌린다.
    avg_cost = 0                      #보통 데이터 가져다가 돌리지만 얘네는 너무 많아, 한방에 못올려 그래서 잘라서 예측시키고 업데이트, 학습 반복 

    for i in range(total_batch):     #배치만큼 돌려
        # training dataset
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)    # 나눠서 돌리는 배치 사이즈

        
        sess.run(optimizer, 
                 feed_dict={x: batch_xs, y: batch_ys, keep_prob: 0.7})    #옵티마이저 러닝, for문만큼
        c = sess.run(cost    #cost 러닝 시켜서 C에 담아
                     , feed_dict={x: batch_xs, y: batch_ys, keep_prob: 0.7})
        
        avg_cost += c / total_batch  #t한 에폭마다 otal batch로 나눠주고 avgcost에 더해줘
        
    print('{:2} : {:}'.format(epoch+1, avg_cost))

total_batch :  550
 1 : 0.3649734963612124
 2 : 0.15779594974761665
 3 : 0.11813766668465998
 4 : 0.09368202495473356
 5 : 0.0805112761411478
 6 : 0.06922437128187577
 7 : 0.062194315552372814
 8 : 0.05793441620232031
 9 : 0.05113876497203649
10 : 0.04904120146720244
11 : 0.04624879586510361
12 : 0.04384695352753623
13 : 0.03991925722538408
14 : 0.03843238693153997
15 : 0.034626799389423646
16 : 0.03514618248562329
17 : 0.03252298609110187
18 : 0.03297787201252176
19 : 0.030040829397198234
20 : 0.02941714020975101


cost가 점점 낮아지넹 업데이트는 total_batch 수 만큼!!

In [12]:
print('{:.4f}'.format(time.time()-start))  #걸린 시간

38.2086


In [13]:
#정확도
pred = tf.equal(tf.argmax(hypo, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(pred, tf.float32))

#[1~3]
print('accuracy :', sess.run(accuracy, feed_dict={x: mnist.test.images, 
                                                  y: mnist.test.labels, keep_prob: 1.0}))

accuracy : 0.9831
